In [1]:
import psycopg2
import csv

In [7]:
# Establishing the connection
conn = psycopg2.connect(
    database="odoo_middleware",
    user='postgres',
    password='12345678',
    host='localhost',
    port='5432'
)
# Connection
cursor = conn.cursor()

In [8]:
def check_if_exist(supplier_code_d):
    exist = "no";
    postgreSQL_select_Query = "select * from supplier where supplier_code=%s"
    cursor.execute(postgreSQL_select_Query, (supplier_code_d,))
    records = cursor.fetchall()
    if(len(records) != 0):
        exist = "yes";
        
    return exist;

def check_name_different(supplier_code_d,supplier_name_d):
    different = "no";
    postgreSQL_select_Query = "select * from supplier where supplier_code=%s"
    cursor.execute(postgreSQL_select_Query, (supplier_code_d,))
    records = cursor.fetchall()
    for m in records:
        if(m[1] != supplier_name_d):
            different = "yes";
        
    return different;

In [9]:
file = open('Master Data Supplier.csv')
csvreader = csv.reader(file)
next(csvreader, None)
rows = []
for row in csvreader:
    #1. Insert: If code not exist
    if(check_if_exist(row[0])) == "no": 
        postgres_insert_query = """ INSERT INTO supplier (supplier_code,supplier_name) VALUES (%s,%s)"""
        record_to_insert = (row[0],row[1])
        cursor.execute(postgres_insert_query, record_to_insert)
        conn.commit()
    #2. Update: If same supplier_code but different name
    else:
        if(check_name_different(row[0],row[1])) == "yes":
            sql_update_query = """Update supplier set supplier_name = %s where supplier_code = %s"""
            cursor.execute(sql_update_query, (row[1], row[0]))
            conn.commit()    
file.close()